In [1]:
import torch 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### 1. 读取数据

In [2]:
data = pd.read_csv('credit-a.csv',header = None)#不要表头
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,30.83,0.000,0,0,9,0,1.25,0,0,1,1,0,202,0.0,-1
1,1,58.67,4.460,0,0,8,1,3.04,0,0,6,1,0,43,560.0,-1
2,1,24.50,0.500,0,0,8,1,1.50,0,1,0,1,0,280,824.0,-1
3,0,27.83,1.540,0,0,9,0,3.75,0,0,5,0,0,100,3.0,-1
4,0,20.17,5.625,0,0,9,0,1.71,0,1,0,1,2,120,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,21.08,10.085,1,1,11,1,1.25,1,1,0,1,0,260,0.0,1
649,1,22.67,0.750,0,0,0,0,2.00,1,0,2,0,0,200,394.0,1
650,1,25.25,13.500,1,1,13,7,2.00,1,0,1,0,0,200,1.0,1
651,0,17.92,0.205,0,0,12,0,0.04,1,1,0,1,0,280,750.0,1


#### 2.多元回归问题，划分输入和输出

In [3]:
X = data.iloc[:,:-1] # iloc取表格的数据[行数据,列数据]，所有的行，列：直到倒数第一列
X = torch.from_numpy(X.values).type(torch.FloatTensor) # 转换数据类型
X

tensor([[0.0000e+00, 3.0830e+01, 0.0000e+00,  ..., 0.0000e+00, 2.0200e+02,
         0.0000e+00],
        [1.0000e+00, 5.8670e+01, 4.4600e+00,  ..., 0.0000e+00, 4.3000e+01,
         5.6000e+02],
        [1.0000e+00, 2.4500e+01, 5.0000e-01,  ..., 0.0000e+00, 2.8000e+02,
         8.2400e+02],
        ...,
        [1.0000e+00, 2.5250e+01, 1.3500e+01,  ..., 0.0000e+00, 2.0000e+02,
         1.0000e+00],
        [0.0000e+00, 1.7920e+01, 2.0500e-01,  ..., 0.0000e+00, 2.8000e+02,
         7.5000e+02],
        [0.0000e+00, 3.5000e+01, 3.3750e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

In [4]:
Y = data.iloc[:,-1:].replace(-1,0) # replace(a,b),将a换成b，二分类问题，0，1
Y = torch.from_numpy(Y.values).type(torch.FloatTensor) # FloatTensor类型


至此，X作为模型的输入，Y作为模型的输出

#### 3. 定义模型

In [5]:
from torch import nn

In [6]:
model = nn.Sequential(
    nn.Linear(15,1),# 输入15，输出1
    nn.Sigmoid() # 激活函数，实现数据的标准化映射
)

In [7]:
model

Sequential(
  (0): Linear(in_features=15, out_features=1, bias=True)
  (1): Sigmoid()
)

In [8]:
lr = 0.0001
opt = torch.optim.SGD(model.parameters(),lr)
loss_fn = nn.BCELoss()

In [9]:
batch_size = 32
it = 0
it = X.size(0) // batch_size # 从0开始
it

20

In [15]:
for epoch in range(100):
    for i in range(it):
        start = i * batch_size
        end = start + batch_size
        
        
        x = X[start:end]
        y = Y[start:end]
        y_predict = model(x) # 预测值
        loss = loss_fn(y_predict,y) #计算损失
        
        # 梯度置0
        opt.zero_grad()
        loss.backward()
        opt.step()
        print('epoch:',epoch,'loss:',loss.data,((model(X).data.numpy() > 0.5).astype('int') == Y.numpy()).mean())
        

epoch: 0 loss: tensor(1.1263) 0.7166921898928025
epoch: 0 loss: tensor(0.7813) 0.6998468606431854
epoch: 0 loss: tensor(1.6626) 0.7243491577335375
epoch: 0 loss: tensor(0.9263) 0.7228177641653905
epoch: 0 loss: tensor(0.6837) 0.7197549770290965
epoch: 0 loss: tensor(0.5807) 0.7090352220520674
epoch: 0 loss: tensor(0.4490) 0.6722817764165391
epoch: 0 loss: tensor(1.8875) 0.6983154670750383
epoch: 0 loss: tensor(6.8921) 0.7335375191424196
epoch: 0 loss: tensor(1.2872) 0.7350689127105666
epoch: 0 loss: tensor(8.1834) 0.7565084226646248
epoch: 0 loss: tensor(0.8589) 0.5987748851454824
epoch: 0 loss: tensor(0.0466) 0.5987748851454824
epoch: 0 loss: tensor(0.1711) 0.5987748851454824
epoch: 0 loss: tensor(22.9528) 0.6049004594180705
epoch: 0 loss: tensor(8.3898) 0.6294027565084227
epoch: 0 loss: tensor(11.6199) 0.6064318529862175
epoch: 0 loss: tensor(0.4002) 0.6033690658499234
epoch: 0 loss: tensor(4.0028) 0.7427258805513017
epoch: 0 loss: tensor(0.7800) 0.7457886676875957
epoch: 1 loss: ten

epoch: 13 loss: tensor(0.2121) 0.664624808575804
epoch: 14 loss: tensor(7.9476) 0.7304747320061256
epoch: 14 loss: tensor(0.3911) 0.655436447166922
epoch: 14 loss: tensor(1.6454) 0.7396630934150077
epoch: 14 loss: tensor(0.7760) 0.7442572741194488
epoch: 14 loss: tensor(0.6438) 0.7366003062787136
epoch: 14 loss: tensor(0.5482) 0.7289433384379785
epoch: 14 loss: tensor(0.4061) 0.6722817764165391
epoch: 14 loss: tensor(1.1734) 0.7243491577335375
epoch: 14 loss: tensor(3.7349) 0.7580398162327718
epoch: 14 loss: tensor(0.6611) 0.7687595712098009
epoch: 14 loss: tensor(1.5221) 0.5957120980091883
epoch: 14 loss: tensor(0.0594) 0.5957120980091883
epoch: 14 loss: tensor(0.0513) 0.5957120980091883
epoch: 14 loss: tensor(0.1793) 0.5941807044410413
epoch: 14 loss: tensor(22.9291) 0.6079632465543645
epoch: 14 loss: tensor(20.0920) 0.6079632465543645
epoch: 14 loss: tensor(33.5698) 0.5604900459418071
epoch: 14 loss: tensor(45.2173) 0.4670750382848392
epoch: 14 loss: tensor(0.7811) 0.609494640122511

epoch: 29 loss: tensor(0.5489) 0.7289433384379785
epoch: 29 loss: tensor(0.4065) 0.6768759571209801
epoch: 29 loss: tensor(1.1106) 0.7274119448698315
epoch: 29 loss: tensor(3.3811) 0.6722817764165391
epoch: 29 loss: tensor(3.8032) 0.7611026033690659
epoch: 29 loss: tensor(3.1800) 0.5957120980091883
epoch: 29 loss: tensor(0.0606) 0.5957120980091883
epoch: 29 loss: tensor(0.0535) 0.5957120980091883
epoch: 29 loss: tensor(0.1808) 0.5957120980091883
epoch: 29 loss: tensor(22.9007) 0.6079632465543645
epoch: 29 loss: tensor(20.0514) 0.6094946401225115
epoch: 29 loss: tensor(33.4826) 0.557427258805513
epoch: 29 loss: tensor(45.1645) 0.4670750382848392
epoch: 29 loss: tensor(0.7897) 0.6094946401225115
epoch: 29 loss: tensor(0.2350) 0.6110260336906586
epoch: 30 loss: tensor(29.3945) 0.5895865237366003
epoch: 30 loss: tensor(22.2476) 0.42879019908116384
epoch: 30 loss: tensor(11.4246) 0.6171516079632465
epoch: 30 loss: tensor(11.6910) 0.77947932618683
epoch: 30 loss: tensor(0.3342) 0.71669218989

epoch: 44 loss: tensor(1.0814) 0.7335375191424196
epoch: 44 loss: tensor(0.7664) 0.7182235834609495
epoch: 44 loss: tensor(2.0769) 0.7565084226646248
epoch: 44 loss: tensor(0.3191) 0.7702909647779479
epoch: 45 loss: tensor(1.2015) 0.7534456355283308
epoch: 45 loss: tensor(0.5209) 0.6707503828483921
epoch: 45 loss: tensor(1.2072) 0.7503828483920367
epoch: 45 loss: tensor(0.7124) 0.7519142419601837
epoch: 45 loss: tensor(0.6036) 0.7565084226646248
epoch: 45 loss: tensor(0.5449) 0.7258805513016845
epoch: 45 loss: tensor(0.3834) 0.667687595712098
epoch: 45 loss: tensor(0.7631) 0.7519142419601837
epoch: 45 loss: tensor(1.3426) 0.6140888208269525
epoch: 45 loss: tensor(6.8541) 0.6110260336906586
epoch: 45 loss: tensor(0.1369) 0.6125574272588055
epoch: 45 loss: tensor(0.0961) 0.6156202143950995
epoch: 45 loss: tensor(0.0903) 0.6110260336906586
epoch: 45 loss: tensor(0.2119) 0.6079632465543645
epoch: 45 loss: tensor(22.6593) 0.6110260336906586
epoch: 45 loss: tensor(11.5447) 0.6156202143950995

epoch: 60 loss: tensor(0.8744) 0.7044410413476263
epoch: 60 loss: tensor(2.3202) 0.7029096477794793
epoch: 60 loss: tensor(1.2647) 0.7090352220520674
epoch: 60 loss: tensor(0.6657) 0.7029096477794793
epoch: 60 loss: tensor(0.5755) 0.7044410413476263
epoch: 60 loss: tensor(0.4380) 0.6738131699846861
epoch: 60 loss: tensor(3.0083) 0.6952526799387443
epoch: 60 loss: tensor(8.7858) 0.7105666156202144
epoch: 60 loss: tensor(2.4514) 0.7105666156202144
epoch: 60 loss: tensor(14.3758) 0.7090352220520674
epoch: 60 loss: tensor(7.3548) 0.7182235834609495
epoch: 60 loss: tensor(1.7946) 0.7197549770290965
epoch: 60 loss: tensor(7.5316) 0.7258805513016845
epoch: 60 loss: tensor(3.9116) 0.7381316998468607
epoch: 60 loss: tensor(2.5444) 0.7381316998468607
epoch: 60 loss: tensor(1.0955) 0.7366003062787136
epoch: 60 loss: tensor(0.7594) 0.7228177641653905
epoch: 60 loss: tensor(2.0730) 0.7549770290964778
epoch: 60 loss: tensor(0.3366) 0.7687595712098009
epoch: 61 loss: tensor(1.1885) 0.7534456355283308

epoch: 75 loss: tensor(1.7766) 0.6983154670750383
epoch: 75 loss: tensor(0.7484) 0.6967840735068913
epoch: 75 loss: tensor(7.7162) 0.7029096477794793
epoch: 75 loss: tensor(4.1038) 0.7090352220520674
epoch: 76 loss: tensor(1.1979) 0.7044410413476263
epoch: 76 loss: tensor(0.8696) 0.7059724349157733
epoch: 76 loss: tensor(2.2506) 0.7044410413476263
epoch: 76 loss: tensor(1.2370) 0.7120980091883614
epoch: 76 loss: tensor(0.6657) 0.7059724349157733
epoch: 76 loss: tensor(0.5739) 0.7059724349157733
epoch: 76 loss: tensor(0.4347) 0.6753445635528331
epoch: 76 loss: tensor(2.8906) 0.6937212863705973
epoch: 76 loss: tensor(8.6894) 0.7105666156202144
epoch: 76 loss: tensor(2.3499) 0.7120980091883614
epoch: 76 loss: tensor(14.1393) 0.7136294027565084
epoch: 76 loss: tensor(7.1395) 0.7197549770290965
epoch: 76 loss: tensor(1.6944) 0.7197549770290965
epoch: 76 loss: tensor(7.2778) 0.7304747320061256
epoch: 76 loss: tensor(3.6745) 0.7411944869831547
epoch: 76 loss: tensor(2.3561) 0.7442572741194488

epoch: 91 loss: tensor(1.2988) 0.7090352220520674
epoch: 91 loss: tensor(0.6620) 0.6998468606431854
epoch: 91 loss: tensor(0.5784) 0.7029096477794793
epoch: 91 loss: tensor(0.4486) 0.6722817764165391
epoch: 91 loss: tensor(3.1479) 0.6921898928024502
epoch: 91 loss: tensor(8.9011) 0.7059724349157733
epoch: 91 loss: tensor(2.5644) 0.7090352220520674
epoch: 91 loss: tensor(14.6461) 0.7105666156202144
epoch: 91 loss: tensor(7.5954) 0.7166921898928025
epoch: 91 loss: tensor(1.9092) 0.7197549770290965
epoch: 91 loss: tensor(7.8228) 0.7197549770290965
epoch: 91 loss: tensor(4.1790) 0.7258805513016845
epoch: 91 loss: tensor(2.7707) 0.7381316998468607
epoch: 91 loss: tensor(1.1123) 0.7350689127105666
epoch: 91 loss: tensor(0.7823) 0.7166921898928025
epoch: 91 loss: tensor(2.3027) 0.7488514548238897
epoch: 91 loss: tensor(0.3984) 0.7611026033690659
epoch: 92 loss: tensor(1.1503) 0.7549770290964778
epoch: 92 loss: tensor(0.5622) 0.6768759571209801
epoch: 92 loss: tensor(1.1710) 0.7549770290964778

In [13]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.1412,  0.0414, -0.0047,  0.0842, -0.0909,  0.0395,  0.2125, -0.2868,
                        0.2438, -0.0333, -0.4784,  0.2579,  0.0171,  0.0135, -0.0348]])),
             ('0.bias', tensor([-0.1219]))])

In [14]:
((model(X).data.numpy() > 0.5).astype('int') == Y.numpy()).mean()

0.7182235834609495